# T-Maze Task

In this notebook, we simulate a T-Maze task (or three-arm bandit task) using an active inference (AIF) agent with the `jax-pymdp` library. Built on JAX, `jax-pymdp` enhances the original [`pymdp` package](https://pymdp-rtd.readthedocs.io/en/latest/) by enabling faster computations and parallel simulations of multiple agents.

<details>
<summary> Click here to see the description of the T-maze task. </summary>

The T-Maze task (generative process) and the agent's generative model we implement in this notebook is adapted from [the sophisticated inference paper](https://discovery.ucl.ac.uk/id/eprint/10124606/). This task represents a classic problem in spatial navigation, learning, and decision-making, where an agent (in this case, a rat) must navigate a T-shaped maze. The agent starts at the centre of the T-maze. Within either the left or right arm, there is either a preferred (i.e., rewarding; cheese) stimulus or an aversive (i.e., punishing; shock) stimulus, with their locations initially unknown to the agent. In the bottom arm, an instructional cue provides information about the location of the rewarding stimulus.

To solve the maze and locate the reward, the agent must decide whether to commit to one of the potentially rewarding arms or first seek information from the cue to identify the more profitable option before taking action. Cue validity represents the probability that the cue correctly indicates the reward's location. An optimal agent should learn to exploit the cue when it is valid and explore the arms when it is not. 
</details>

<details>
<summary> Click here to see the setup requirements. </summary>
To run this notebook, it is recommended to set up and use a virtual environment. You can refer to <a href="https://packaging.python.org/en/latest/guides/installing-using-pip-and-virtual-environments/">this guide</a> to set up a virtual environment.

Then, install the current repo (v.1.0.0_alpha), which contains the `jax-pymdp` package, as a package in editable mode by running the following command in your terminal:
```bash
pip install -e .
```
Note: While `jax` and `jaxlib` are included as dependencies when you install the current repo, it is strongly recommended to manually install JAX tailored to your system (e.g., for GPU support). Follow the <a href="https://jax.readthedocs.io/en/latest/installation.html">JAX installation guide</a> to ensure compatibility with your hardware.
</details>

### In this notebook, we will create the following simulations: 
1. A deterministic generative process (environment), and a single agent solving the task with vanilla active inference.
2. A noisy generative process, and a single agent solving the task with vanilla active inference.
3. A noisy generative model with A and B learning, with correct prior structure of those parameters, and a single agent solving the task with vanilla active inference.
- A learning only
- B learning only
- A and B learning
4. A noisy generative model with A and B learning, with incorrect prior structure of those parameters, and a single agent solving the task with vanilla active inference.
- A learning only
- B learning only
- A and B learning
5. A deterministic generative process, and a single agent solving the task with sophisticated inference
6. A deterministic generative process, and a single agent solving the task with inductive inference
7. A deterministic generative process, and multiple agents solving the task with vanilla active inference
8. A deterministic generative process, and multiple agents solving the task with sophisticated inference


In [1]:
# a way to edit and run code and see the effects in the notebook without having to restart the kernel
%load_ext autoreload
%autoreload 2

# importing necessary libraries
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
import mediapy

from jax import random as jr
from pymdp.envs import TMaze, rollout
from pymdp.agent import Agent
from PIL import Image

### Creating the T-Maze Task (Generative Process)

- The `batch_size` parameter specifies the number of environments to run in parallel.
- The `reward_condition` parameter determines the reward location: `0` for the left arm, `1` for the right arm, or `None` for random allocation.
- The `reward_probability` parameter sets the chance of receiving a reward in the correct arm. For example, if set at 0.80, there would be an 80% chance of reward and 20% chance of no outcome in the rewarding arm.
- The `punishment_probability` parameter specifies the likelihood of punishment in the other arm. For example, if set at 0.80, there would be an 80% chance of punishment and 20% chance of no outcome in the non-rewarding arm.
- The `cue_validity` parameter represents the accuracy of the cues as a probability between 0 and 1.

<details>
<summary> Click here to see how the generative process is set up. </summary>

#### States and Observations

**State Factors:**
1. Location (5 states):
    - 0: centre (start location)
    - 1: left arm
    - 2: right arm
    - 3: cue location (bottom arm)
    - 4: middle of arms (between left and right arm)
2. Reward Location (2 states):
    - 0: reward in left arm 
    - 1: reward in right arm

**Observation Modalities:**
1. Location (5 observations):
    - Matches the location states exactly
2. Outcome (3 observations):
    - 0: no outcome
    - 1: reward (cheese)
    - 2: punishment (shock)
3. Cue (3 observations):
    - 0: no cue
    - 1: left arm cued
    - 2: right arm cued

#### Environment Parameters

**Observation Likelihood Model (A):**
- A[0]: Location observations (5x5x1 tensor)
  - Perfect mapping between true and observed location.
- A[1]: Outcome observations (3x5x2 tensor)
  - In the more rewarding arm, reward is presented with a likelihood determined by the `reward_probability` parameter.
  - In the less rewarding arm, punishment is presented with a likelihood determined by the `punishment_probability` parameter.
  - No outcomes are observed in the centre/start location, cue location, or middle of the arms.
- A[2]: Cue observations (3x5x2 tensor)
  - Indicating the reward location, at the cue location (bottom arm), with accuracy set by the `cue_validity` parameter.
  - No cues visible elsewhere.

**Transition Model (B):**
- B[0]: Location transitions (5x5x5 tensor)
  - Agent can move between adjacent maze cells or stay in the same cell.
- B[1]: Reward location (2x2x1 tensor)
  - Reward location remains fixed throughout trial.

**Initial Conditions (D):**
- D[0]: Starting location (5x1 tensor)
  - Agent always begins in centre location
- D[1]: Reward placement (2x1 tensor)
  - Default: Equal chance (50/50) of reward in either arm (`reward_condition=None`)
  - Optional: Can fix reward to specific arm, by setting `reward_condition` to `0` (for left arm) or `1` (for right arm)

</details>



In [2]:
# setting the parameters for the environment
batch_size = 1 # number of environments to run in parallel
reward_condition = None # 0 is reward in left arm, 1 is reward in right arm, None is random allocation
reward_probability = 1.0 # 100% chance of reward in the correct arm
punishment_probability = 1.0 # 100% chance of punishment in the other arm
cue_validity = 1.0 # 100% valid cues

# initialising the environment. see tmaze.py in pymdp/envs for the implementation details.
env = TMaze( 
    batch_size=batch_size, 
    reward_probability=reward_probability,     
    punishment_probability=punishment_probability, 
    cue_validity=cue_validity,          
    reward_condition=reward_condition   
)

# you may print the environment parameters to see the shapes of the tensors and the values by editing and uncommenting the following lines and running the code: 

# print([a.shape for a in env.params["A"]]) # shape of all A tensors; the shape should start with the batch_size, then the rows, columns, and additional dimensions for the dependencies
# print(env.params["A"][1][0][:,:,1]) # likelihood of observing no outcome, reward, or punishment (rows), in each location (columns), when the reward condition is 1 (right arm)
# print(env.params["A"][2][0][:,:,0]) # likelihood of observing no cue, left arm cued, or right arm cued (rows), in each location (columns), when the reward condition is 0 (left arm)

# print([b.shape for b in env.params["B"]]) # shape of all B tensors
# print(env.params["B"][0][0][:,:,4]) # probability of transitioning to each location (rows), from each location (columns), when the agent wants to move to the middle of the arms (location 4)

# 1. A deterministic generative process (environment), and a single agent solving the task with vanilla active inference.

### Creating the Agent (Generative Model)

We will create the agent's generative model based off the A and B tensors of the environment. The A and B tensors remain the same as, in this simple design, we assume the agent has knowledge of the environment's parameters - i.e., it knows that the likelihood of observing a reward in the left arm is 1.0 if the reward is actually in the left arm (`reward_condition=0`), and it knows the cues are 100% accurate (`cue_validity=1.0`), and it knows that the reward location will be fixed throughout the trial (not volatile environment). We can of course change these assumptions to create a more complex agent, and we will do that in the next sections where the environment will be more stochastic and we will also add noise to the agent's generative model.

The preference tensors (C) are set using the A tensor's shape. The agent is set to not have any preference to observe certain locations and cues. The agent is set to prefer reward and avoid punishment.

The initial beliefs tensors (D; i.e., priors) are set using the B tensor's shape. The agent has a prior to  start in the centre location and the reward location is uniformly distributed.  

In [3]:
#  setting A tensors from the environment parameters
A = [jnp.array(a, dtype=jnp.float32) for a in env.params["A"]]
A_dependencies = env.dependencies["A"] # dependencies allow you to specify the state factors the observation modality depends on so you dont have to compute the full tensor using all state factors

# setting B tensors from the environment parameters
B = [jnp.array(b, dtype=jnp.float32) for b in env.params["B"]]
B_dependencies = env.dependencies["B"]

# creating C tensors filled with zeros for [location], [reward], [cue] based on A shapes
C = [jnp.zeros((batch_size, a.shape[1]), dtype=jnp.float32) for a in A] 
# setting preferences for outcomes only
C[1] = C[1].at[:,1].set(2.0)    # prefer reward
C[1] = C[1].at[:,2].set(-3.0)   # avoid punishment


# creating D tensors [location], [reward] based on B shapes
D = []
# D[0]: location - all zeros except location 0 (centre) because the agent always starts in the centre
D_loc = jnp.zeros((batch_size, B[0].shape[1]), dtype=jnp.float32) 
D_loc = D_loc.at[0,0].set(1.0)  # set centre location to 1.0
D.append(D_loc)

# D[1]: reward location - uniform distribution
D_reward = jnp.ones((batch_size, B[1].shape[1]), dtype=jnp.float32) 
D_reward = D_reward / jnp.sum(D_reward, axis=1, keepdims=True)  # normalise to get uniform distribution
D.append(D_reward)


# initialising the agent
agent = Agent(
    A, B, C, D, 
    policy_len=5, # how long the action sequence is that the agent is evaluating
    A_dependencies=A_dependencies, 
    B_dependencies=B_dependencies,
    apply_batch=False,
    learn_A=False,
    learn_B=False
)

# you may print the agent's generative model parameters to see the shapes of the tensors and the values by editing and uncommenting the following lines and running the code: 

# print([a.shape for a in agent.A]) # shape of all A tensors
# print(agent.A[1][0][:,:,1]) # likelihood of observing no outcome, reward, or punishment (rows), in each location (columns), when the reward condition is 1 (right arm)
# print(agent.C[1]) # preferences for outcomes

In [4]:
key = jr.PRNGKey(0) # random key for the aif loop
T = 10 # number of timesteps to rollout the aif loop for
_, info, _ = rollout(agent, env, num_timesteps=T, rng_key=key) # running the aif loop

# you may print the info dictionary to see the numerical results of the aif agent completing the T-maze task by editing and uncommenting the following lines and running the code: 

# print(info.keys()) # keys in the info dictionary
# print(info["action"][:,0,:]) # actions taken by the agent (locations throughout the maze) 
# print(info["observation"][0]) # observations of the locations for each batch
# print(info["observation"][2]) # observations of the cues for each batch
# print(jnp.around(info["qs"][1], decimals=2)) # posterior beliefs about the reward location
# print(info["qpi"][0].shape) # shape of the policy tensor


#### Rendering the Task to Visualise the Agent's Behaviour

In [5]:
frames = []
for t in range(info["observation"][0].shape[0]):  # iterate over timesteps
    # get observations for this timestep
    observations_t = [
        info["observation"][0][t, :, :],
        info["observation"][1][t, :, :],  
        info["observation"][2][t, :, :]   
    ]
       
    frame = env.render(mode="rgb_array", observations=observations_t) # render the environment using the observations for this timestep
    frame = np.asarray(frame, dtype=np.uint8)
    plt.close()  # close the figure to prevent memory leak
    frames.append(frame)

frames = np.array(frames, dtype=np.uint8)
mediapy.show_video(frames, fps=1)

# uncomment the following lines to save the video as a gif
# pil_frames = [Image.fromarray(frame) for frame in frames]
# reward_location = "random" if reward_condition is None else ("left" if reward_condition == 0 else "right")
# filename = f"tmaze_{batch_size}_{reward_location}.gif"
# pil_frames[0].save(
#     filename,
#     save_all=True,
#     append_images=pil_frames[1:],
#     duration=1000,  # 1000ms per frame
#     loop=0
# )

# 2. A noisy generative process, and a single agent solving the task with vanilla active inference.

In [6]:
# THE GENERATIVE PROCESS (NOISY)
# setting the parameters for the environment
batch_size = 4 # number of environments to run in parallel
reward_condition = None # 0 is reward in left arm, 1 is reward in right arm, None is random allocation
reward_probability = 0.7 # 70% chance of reward in the correct arm
punishment_probability = 0.6 # 60% chance of punishment in the other arm
cue_validity = 0.9 # 90% valid cues

# initialising the environment. see tmaze.py in pymdp/envs for the implementation details.
env = TMaze( 
    batch_size=batch_size, 
    reward_probability=reward_probability,     
    punishment_probability=punishment_probability, 
    cue_validity=cue_validity,          
    reward_condition=reward_condition   
)


In [7]:
# THE GENERATIVE MODEL
#  setting A tensors from the environment parameters
A = [jnp.array(a, dtype=jnp.float32) for a in env.params["A"]]
A_dependencies = env.dependencies["A"] # dependencies allow you to specify the state factors the observation modality depends on so you dont have to compute the full tensor using all state factors

# setting B tensors from the environment parameters
B = [jnp.array(b, dtype=jnp.float32) for b in env.params["B"]]
B_dependencies = env.dependencies["B"]

# creating C tensors filled with zeros for [location], [reward], [cue] based on A shapes
C = [jnp.zeros((batch_size, a.shape[1]), dtype=jnp.float32) for a in A] 
# setting preferences for outcomes only
C[1] = C[1].at[:,1].set(2.0)    # prefer reward
C[1] = C[1].at[:,2].set(-3.0)   # avoid punishment


# creating D tensors [location], [reward] based on B shapes
D = []
# D[0]: location - all zeros except location 0 (centre) because the agent always starts in the centre
D_loc = jnp.zeros((batch_size, B[0].shape[1]), dtype=jnp.float32) 
D_loc = D_loc.at[0,0].set(1.0)  # set centre location to 1.0
D.append(D_loc)

# D[1]: reward location - uniform distribution
D_reward = jnp.ones((batch_size, B[1].shape[1]), dtype=jnp.float32) 
D_reward = D_reward / jnp.sum(D_reward, axis=1, keepdims=True)  # normalise to get uniform distribution
D.append(D_reward)


# initialising the agent
agent = Agent(
    A, B, C, D, 
    policy_len=5, # how long the action sequence is that the agent is evaluating
    A_dependencies=A_dependencies, 
    B_dependencies=B_dependencies,
    apply_batch=False,
    learn_A=False,
    learn_B=False
)

key = jr.PRNGKey(0) # random key for the aif loop
T = 10 # number of timesteps to rollout the aif loop for
_, info, _ = rollout(agent, env, num_timesteps=T, rng_key=key) # running the aif loop


In [8]:
frames = []
for t in range(info["observation"][0].shape[0]):  # iterate over timesteps
    # get observations for this timestep
    observations_t = [
        info["observation"][0][t, :, :],
        info["observation"][1][t, :, :],  
        info["observation"][2][t, :, :]   
    ]
       
    frame = env.render(mode="rgb_array", observations=observations_t) # render the environment using the observations for this timestep
    frame = np.asarray(frame, dtype=np.uint8)
    plt.close()  # close the figure to prevent memory leak
    frames.append(frame)

frames = np.array(frames, dtype=np.uint8)
mediapy.show_video(frames, fps=1)

This browser does not support the video tag.


# A and B learning

### loading the generative process (environment)

In [9]:
# setting the parameters for the environment
batch_size = 1 # number of environments to run in parallel
reward_condition = 1 # 0 is reward in left arm, 1 is reward in right arm, None is random allocation
reward_probability = 1.0 # 100% chance of reward in the correct arm
punishment_probability = 1.0 # 100% chance of punishment in the other arm
cue_validity = 1.0 # 100% valid cues

# initialising the environment. see tmaze.py in pymdp/envs for the implementation details.
env = TMaze( 
    batch_size=batch_size, 
    reward_probability=reward_probability,     
    punishment_probability=punishment_probability, 
    cue_validity=cue_validity,          
    reward_condition=reward_condition   
)

### the agent's generative model

In [10]:
#  setting A tensors from the environment parameters
A = [jnp.array(a, dtype=jnp.float32) for a in env.params["A"]]
A_dependencies = env.dependencies["A"] 

# adding noise to flatten the distributions (make more uncertain) 
noise_level = 0.3 
for i in [1, 2]:  # only modifying the outcome (i=1) and cue (i=2) observation likelihood mappings
    # A[i] = jnp.flip(A[i], axis=1) # flipping for testing purposes

    noise = noise_level * jnp.ones_like(A[i])
    A[i] = A[i] + noise
    A[i] = A[i] / jnp.sum(A[i], axis=1, keepdims=True) # normalise to ensure each distribution sums to 1

pA = A

# setting B tensors from the environment parameters
B = [jnp.array(b, dtype=jnp.float32) for b in env.params["B"]]
B_dependencies = env.dependencies["B"]

# B[0] = jnp.flip(B[0], axis=(1,2)) # flipping for testing purposes

# adding noise to flatten the distributions (make more uncertain) 
key, subkey = jr.split(key)
noise = noise_level * jr.uniform(subkey, shape=B[0].shape)
B[0] = B[0] + noise
B[0] = B[0] / jnp.sum(B[0], axis=1, keepdims=True) # normalise to ensure each distribution sums to 1

key, subkey = jr.split(key)
noise = noise_level * jr.uniform(subkey, shape=B[1].shape)
# B[1] = jnp.flip(B[1], axis=(1,2)) # flipping for testing purposes
B[1] = B[1] + noise
B[1] = B[1] / jnp.sum(B[1], axis=1, keepdims=True) # normalise to ensure each distribution sums to 1

pB = B

# creating C tensors filled with zeros for [location], [reward], [cue] based on A shapes
C = [jnp.zeros((batch_size, a.shape[1]), dtype=jnp.float32) for a in A] 
# setting preferences for outcomes only
C[1] = C[1].at[:,1].set(2.0)    # prefer reward
C[1] = C[1].at[:,2].set(-3.0)   # avoid punishment


# creating D tensors [location], [reward] based on B shapes
D = []
# D[0]: location - all zeros except location 0 (centre) because the agent always starts in the centre
D_loc = jnp.zeros((batch_size, B[0].shape[1]), dtype=jnp.float32) 
D_loc = D_loc.at[0,0].set(1.0)  # set centre location to 1.0
D.append(D_loc)

# D[1]: reward location - uniform distribution
D_reward = jnp.ones((batch_size, B[1].shape[1]), dtype=jnp.float32) 
D_reward = D_reward / jnp.sum(D_reward, axis=1, keepdims=True)  # normalise to get uniform distribution
D.append(D_reward)


# initialising the agent
agent = Agent(
    A, B, C, D, 
    pA=pA,
    pB=pB, # adding the noisy A tensor for learning
    policy_len=5, # how long the action sequence is that the agent is evaluating
    A_dependencies=A_dependencies, 
    B_dependencies=B_dependencies,
    apply_batch=False, 
    learn_A=True,
    learn_B=True,
    gamma=0.1,
    action_selection="stochastic"
)

In [11]:
key = jr.PRNGKey(0) # random key for the aif loop
T = 10 # number of timesteps to rollout the aif loop for
_, info, _ = rollout(agent, env, num_timesteps=T, rng_key=key) # running the aif loop

In [12]:
print("at t=0")
print(info["agent"].B[0][0,0,:,:,1])
print()
print(info["agent"].B[1][0,0,:,:,1])



at t=0
[[0.55928385 0.03319283 0.0423363  0.14858027 0.00967858]
 [0.12059184 0.82310116 0.11142875 0.04356447 0.80422693]
 [0.05870756 0.01633091 0.81928074 0.07423069 0.05903195]
 [0.14976892 0.07099813 0.02205921 0.72776103 0.12234119]
 [0.11164781 0.05637698 0.00489496 0.00586354 0.00472126]]

[[0.812142   0.1745717 ]
 [0.18785803 0.8254283 ]]


In [13]:
print("at t=10")
print(info["agent"].B[0][-1,0,:,:,1])
print()
print(info["agent"].B[1][-1,0,:,:,1])


at t=10
[[0.7796419  0.03319283 0.0423363  0.14858027 0.00483929]
 [0.06029592 0.82310116 0.11142875 0.04356447 0.9021135 ]
 [0.02935378 0.01633091 0.81928074 0.07423069 0.02951598]
 [0.07488446 0.07099813 0.02205921 0.72776103 0.0611706 ]
 [0.0558239  0.05637698 0.00489496 0.00586354 0.00236063]]

[[0.47945264 0.36392748]
 [0.5205474  0.63607246]]


In [14]:
print("the environment's A tensor")
print(env.params["A"][1][0][:,:,1])
print(env.params["A"][2][0][:,:,1])

the environment's A tensor
[[1. 0. 0. 1. 1.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]]
[[1. 1. 1. 0. 1.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]]


In [15]:
print("at t=0")
print(agent.A[1][0][:,:,1])
print()
print(agent.A[2][0][:,:,1])



at t=0
[[0.68421054 0.15789475 0.15789476 0.68421054 0.68421054]
 [0.15789476 0.15789475 0.68421054 0.15789476 0.15789476]
 [0.15789476 0.68421054 0.15789476 0.15789476 0.15789476]]

[[0.68421054 0.68421054 0.68421054 0.15789475 0.68421054]
 [0.15789476 0.15789476 0.15789476 0.15789475 0.15789476]
 [0.15789476 0.15789476 0.15789476 0.68421054 0.15789476]]


In [16]:
print("at t=10")
print(info["agent"].A[1][-1,0,:,:,1])
print()
print(info["agent"].A[2][-1,0,:,:,1])


at t=10
[[0.8974463  0.08694159 0.08543371 0.68421054 0.9030858 ]
 [0.05127685 0.08694159 0.8291326  0.15789476 0.04845705]
 [0.05127685 0.8261168  0.08543371 0.15789476 0.04845705]]

[[0.8974463  0.8261168  0.8291326  0.15789475 0.9030858 ]
 [0.05127685 0.08694161 0.08543371 0.15789475 0.04845705]
 [0.05127685 0.08694161 0.08543371 0.68421054 0.04845705]]


In [17]:
print("at t=10")
print(info["agent"].pA[1][-1,0,:,:,1])
print(info["agent"].pA[2][-1,0,:,:,1])

at t=10
[[2.7634706  0.15789475 0.15789476 0.68421054 2.9426582 ]
 [0.15789476 0.15789475 1.5323657  0.15789476 0.15789476]
 [0.15789476 1.5003119  0.15789476 0.15789476 0.15789476]]
[[2.7634706  1.5003119  1.5323657  0.15789475 2.9426582 ]
 [0.15789476 0.15789476 0.15789476 0.15789475 0.15789476]
 [0.15789476 0.15789476 0.15789476 0.68421054 0.15789476]]


In [18]:
print(info["observation"][0][:, 1, :])



[[0.]
 [0.]
 [0.]
 [4.]
 [1.]
 [4.]
 [0.]
 [4.]
 [4.]
 [2.]
 [2.]]


In [19]:
print(agent.gamma)

[0.1]


In [20]:
frames = []
for t in range(info["observation"][0].shape[0]):  # iterate over timesteps
    # get observations for this timestep
    observations_t = [
        info["observation"][0][t, :, :],
        info["observation"][1][t, :, :],  
        info["observation"][2][t, :, :]   
    ]
       
    frame = env.render(mode="rgb_array", observations=observations_t) # render the environment using the observations for this timestep
    frame = np.asarray(frame, dtype=np.uint8)
    plt.close()  # close the figure to prevent memory leak
    frames.append(frame)

frames = np.array(frames, dtype=np.uint8)
mediapy.show_video(frames, fps=1)

This browser does not support the video tag.


In [21]:
print(info["action"])

[[[0 0]]

 [[2 0]]

 [[1 0]]

 [[4 0]]

 [[1 0]]

 [[4 0]]

 [[0 0]]

 [[4 0]]

 [[4 0]]

 [[2 0]]

 [[2 0]]]


In [22]:
# Random initialization test
key = jr.PRNGKey(42)

# Get base tensors from environment
A = [jnp.array(a, dtype=jnp.float32) for a in env.params["A"]]
A_dependencies = env.dependencies["A"]

# Randomize A tensors for outcome and cue observations
for i in [1, 2]:
    key, subkey = jr.split(key)
    # Generate random values between 0 and 1
    A[i] = jr.uniform(subkey, shape=A[i].shape)
    # Normalize to ensure valid probability distributions
    A[i] = A[i] / jnp.sum(A[i], axis=1, keepdims=True)

pA = A  # Store randomized A tensors for learning

# Get base B tensors
B = [jnp.array(b, dtype=jnp.float32) for b in env.params["B"]]
B_dependencies = env.dependencies["B"]

# Randomize location transitions
key, subkey = jr.split(key)
B[0] = jr.uniform(subkey, shape=B[0].shape)
B[0] = B[0] / jnp.sum(B[0], axis=1, keepdims=True)

# Randomize reward location transitions
key, subkey = jr.split(key)
B[1] = jr.uniform(subkey, shape=B[1].shape)
B[1] = B[1] / jnp.sum(B[1], axis=1, keepdims=True)

pB = B  # Store randomized B tensors for learning

# Initialize agent with randomized tensors
random_agent = Agent(
    A, B, C, D,
    pA=pA,
    pB=pB,
    policy_len=5,
    A_dependencies=A_dependencies,
    B_dependencies=B_dependencies,
    apply_batch=False,
    learn_A=True,
    learn_B=True,
    gamma=0.1,  # You might want to adjust learning rate
    action_selection="stochastic"
)

# Run simulation
key = jr.PRNGKey(0)
T = 40  # Increased timesteps to give more learning opportunity
_, info, _ = rollout(random_agent, env, num_timesteps=T, rng_key=key)

In [23]:
print("the environment's A tensor")
print(env.params["A"][1][0][:,:,1])
print(env.params["A"][2][0][:,:,1])

the environment's A tensor
[[1. 0. 0. 1. 1.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]]
[[1. 1. 1. 0. 1.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]]


In [24]:
# Check initial A tensors
print("Initial outcome observations (A[1]):")
print(random_agent.A[1][0,:,:,1])
print("\nInitial cue observations (A[2]):")
print(random_agent.A[2][0,:,:,1])

# Check final A tensors
print("\nFinal outcome observations:")
print(info["agent"].A[1][-1,0,:,:,1])
print("\nFinal cue observations:")
print(info["agent"].A[2][-1,0,:,:,1])

Initial outcome observations (A[1]):
[[0.5218738  0.30091077 0.4679604  0.14330989 0.16058761]
 [0.22740644 0.18828125 0.367972   0.15851246 0.43297487]
 [0.25071982 0.51080793 0.16406764 0.69817764 0.40643755]]

Initial cue observations (A[2]):
[[0.6556678  0.12907703 0.24858476 0.04353992 0.07885107]
 [0.33983567 0.22772244 0.34716552 0.59050167 0.71792424]
 [0.00449653 0.6432005  0.40424973 0.36595842 0.20322472]]

Final outcome observations:
[[0.9631952  0.04830016 0.05298445 0.8754568  0.88928324]
 [0.0175051  0.03022163 0.92843914 0.02304408 0.0571085 ]
 [0.0192997  0.9214782  0.01857643 0.10149904 0.05360828]]

Final cue observations:
[[9.7349423e-01 8.6020535e-01 9.1492158e-01 6.3297064e-03 8.7850237e-01]
 [2.6159586e-02 3.6552463e-02 3.9307546e-02 8.5845418e-02 9.4692744e-02]
 [3.4612996e-04 1.0324219e-01 4.5770861e-02 9.0782487e-01 2.6804926e-02]]


In [25]:
# Check initial B tensors
print("Initial outcome observations (B[0]):")
print(random_agent.B[0][0,:,:,1])
print("\nInitial cue observations (B[1]):")
print(random_agent.B[1][0,:,:,1])

# Check final B tensors
print("\nFinal outcome observations:")
print(info["agent"].B[0][-1,0,:,:,1])
print("\nFinal cue observations:")
print(info["agent"].B[1][-1,0,:,:,1])
print("\nEnvironment's B tensor")
print(env.params["B"][0][0][:,:,1])
print(env.params["B"][1][0][:,:,1])


Initial outcome observations (B[0]):
[[0.09340456 0.10412216 0.08499063 0.26434857 0.25719061]
 [0.09820656 0.08211101 0.01237284 0.331993   0.22598736]
 [0.25686646 0.3196498  0.4598286  0.08734377 0.09786513]
 [0.0134291  0.24751367 0.26943    0.07691143 0.01194376]
 [0.5380933  0.24660337 0.17337795 0.23940317 0.40701312]]

Initial cue observations (B[1]):
[[0.2631618  0.18744703]
 [0.73683816 0.812553  ]]

Final outcome observations:
[[0.77335113 0.05206108 0.04249531 0.26434857 0.0857302 ]
 [0.02455164 0.5410555  0.00618642 0.331993   0.74199575]
 [0.06421661 0.1598249  0.7299143  0.08734377 0.03262171]
 [0.00335727 0.12375683 0.134715   0.07691143 0.00398125]
 [0.13452333 0.12330168 0.08668897 0.23940317 0.13567103]]

Final cue observations:
[[0.15513998 0.06100335]
 [0.84485996 0.9389967 ]]

Environment's B tensor
[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 1.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0.]]
[[1. 0.]
 [0. 1.]]


In [26]:
frames = []
for t in range(info["observation"][0].shape[0]):  # iterate over timesteps
    # get observations for this timestep
    observations_t = [
        info["observation"][0][t, :, :],
        info["observation"][1][t, :, :],  
        info["observation"][2][t, :, :]   
    ]
       
    frame = env.render(mode="rgb_array", observations=observations_t) # render the environment using the observations for this timestep
    frame = np.asarray(frame, dtype=np.uint8)
    plt.close()  # close the figure to prevent memory leak
    frames.append(frame)

frames = np.array(frames, dtype=np.uint8)
mediapy.show_video(frames, fps=1)

This browser does not support the video tag.
